In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
html_doc = """
<html><head><title>Web scraping pro datovou analýzu v Pythonu - sedlakovi.org</title></head>
<body>
<p class="title"><b>Web scraping pro datovou analýzu v Pythonu</b></p>

<p class="description">
    Náš kurz Vás naučí jak lze snadno získávat data z webových stránek, 
    které můžete používat k následné analýze anebo reportingu. 
    Příkladem může být analýza kurzu měn, recenzí kaváren nebo cen zboží. 
    Náš kurz Vám ukáže celý postup od identifikace datového zdroje, 
    navržení vhodného scrapingu a přípravy dat pro následnou analýzu. 
</p>

<p class="description">
    Naučíte se:
    <ul>
        <li>Strukturu HTML</li>
        <li>Jak získat obsah webových stránek (knihovna requests)</li>
        <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>
        <li>Jak následně data zpracovat (knihovna pandas)</li>
    </ul>
</p>

<table>
<tr> <th class="table-header"> Naučím se pracovat s: </th>
<td class="table-data">
<a href="http://docs.python-requests.org/en/master/" class="library" id="link1">Requests</a>,
<a href="https://www.crummy.com/software/BeautifulSoup/"  class="library" id="link2">BeautifulSoup</a>,
<a href="https://pandas.pydata.org/" class="library" id="link3">Pandas</a>
</td>
</tr>
</table>
"""

In [3]:
soup = BeautifulSoup(html_doc, 'html.parser')

# Cvičení 3.1

Vyberte odstavec s třídou `title`.

`Web scraping pro datovou analýzu v Pythonu`

In [4]:
soup.find(class_='title').text

'Web scraping pro datovou analýzu v Pythonu'

In [5]:
soup.find(attrs={'class': 'title'}).text

'Web scraping pro datovou analýzu v Pythonu'

In [6]:
soup.find(name='p', attrs={'class': 'title'}).text

'Web scraping pro datovou analýzu v Pythonu'

# Cvičení 3.2

Vyberte všechny položky seznamu

```
 <li>Strukturu HTML</li>
 <li>Jak získat obsah webových stránek (knihovna requests)</li>
 <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>
 <li>Jak následně data zpracovat (knihovna pandas)</li>
```

** Bonus:** zbavte se HTML tagů (např. `Jak získat obsah webových stránek (knihovna requests)`). 

In [7]:
soup.find_all('li')

[<li>Strukturu HTML</li>,
 <li>Jak získat obsah webových stránek (knihovna requests)</li>,
 <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>,
 <li>Jak následně data zpracovat (knihovna pandas)</li>]

In [8]:
[li.text for li in soup.find_all('li')]

['Strukturu HTML',
 'Jak získat obsah webových stránek (knihovna requests)',
 'Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)',
 'Jak následně data zpracovat (knihovna pandas)']

# Cvičení 5.1

Zjistěte na stránkach `https://www.seznam.cz/`, kdo má dnes svátek.

In [9]:
url = 'https://www.seznam.cz/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
soup.find(class_='name-link').text

'Klára'

# Cvičení 5.2

Najděte které filmy se promítaly 8.8.2019 v kině Aero (první v seznamu abecedně seřazených kinech).

`Je mi fajn s.r.o.`

In [10]:
url = 'https://dokina.tiscali.cz/program-kin?date=2019-08-08&cinemasType=nonmulti&tab=cinemas&cinemasOrdering=title'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [11]:
program = soup.select_one('.program')

In [12]:
program.select_one('.cinema-info .name').text

'Aero'

In [13]:
program.select_one('.program-table .name').text

'Je mi fajn s.r.o.'